In [1]:
import pandas as pd
import numpy as np
import wfdb
import ast

from scipy import stats
from scipy.signal import butter, lfilter

# Fectching diagnostic data 

In [5]:
chapman = pd.read_excel("data/chapman/Diagnostics.xlsx")
chapman

,FileName,Rhythm,Beat,PatientAge,Gender,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,RAxis,TAxis,QRSCount,QOnset,QOffset,TOffset
0,MUSE_20180113_171327_27000,AFIB,RBBB TWC,85,MALE,117,234,114,356,496,81,-27,19,208,265,386
1,MUSE_20180112_073319_29000,SB,TWC,59,FEMALE,52,52,92,432,401,76,42,8,215,261,431
2,MUSE_20180111_165520_97000,SA,NONE,20,FEMALE,67,67,82,382,403,88,20,11,224,265,415
3,MUSE_20180113_121940_44000,SB,NONE,66,MALE,53,53,96,456,427,34,3,9,219,267,447
4,MUSE_20180112_122850_57000,AF,STDD STTC,73,FEMALE,162,162,114,252,413,68,-40,26,228,285,354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10641,MUSE_20181222_204306_99000,SVT,NONE,80,FEMALE,196,73,168,284,513,258,244,32,177,261,319
10642,MUSE_20181222_204309_22000,SVT,NONE,81,FEMALE,162,81,162,294,482,110,-75,27,173,254,320
10643,MUSE_20181222_204310_31000,SVT,NONE,39,MALE,152,92,152,340,540,250,38,25,208,284,378
10644,MUSE_20181222_204312_58000,SVT,NONE,76,MALE,175,178,128,310,529,98,-83,29,205,269,360


In [6]:
chapman['Rhythm'].value_counts()

Rhythm
SB       3889
SR       1826
AFIB     1780
ST       1568
SVT       587
AF        445
SA        399
AT        121
AVNRT      16
AVRT        8
SAAWR       7
Name: count, dtype: int64

In [7]:
print(list(chapman))

['FileName', 'Rhythm', 'Beat', 'PatientAge', 'Gender', 'VentricularRate', 'AtrialRate', 'QRSDuration', 'QTInterval', 'QTCorrected', 'RAxis', 'TAxis', 'QRSCount', 'QOnset', 'QOffset', 'TOffset']


In [8]:
# drop unnecessary columns and rename remaining columns to match PBT-XL database
chapman.drop(columns=['Beat','VentricularRate', 'AtrialRate', 
                      'QRSDuration', 'QTInterval', 'QTCorrected', 'RAxis', 'TAxis', 
                      'QRSCount', 'QOnset', 'QOffset', 'TOffset'], inplace=True)
chapman.rename(columns={'FileName':'filename', 'Rhythm':'rhythm', 'PatientAge':'age',
                        'Gender':'sex'}, inplace=True)

In [9]:
chapman.head()

,filename,rhythm,age,sex
0,MUSE_20180113_171327_27000,AFIB,85,MALE
1,MUSE_20180112_073319_29000,SB,59,FEMALE
2,MUSE_20180111_165520_97000,SA,20,FEMALE
3,MUSE_20180113_121940_44000,SB,66,MALE
4,MUSE_20180112_122850_57000,AF,73,FEMALE


In [10]:
# check for null values
chapman.isnull().values.any()

False

In [11]:
chapman['rhythm'].value_counts()

rhythm
SB       3889
SR       1826
AFIB     1780
ST       1568
SVT       587
AF        445
SA        399
AT        121
AVNRT      16
AVRT        8
SAAWR       7
Name: count, dtype: int64

In [12]:
# change rhythm code for sinus irregularity from SA to SI
chapman['rhythm'].replace({'SA': 'SI'}, inplace=True)
chapman['rhythm'].value_counts()

rhythm
SB       3889
SR       1826
AFIB     1780
ST       1568
SVT       587
AF        445
SI        399
AT        121
AVNRT      16
AVRT        8
SAAWR       7
Name: count, dtype: int64

In [13]:
# change male to 0 and female to 1
chapman['sex'].replace({'MALE': 0, 'FEMALE':1}, inplace=True)

# Helper Functions

### Butterworth Filter

In [14]:
def butter_bandpass(lowcut, highcut, frequency, order):
    nyquist_freq = 0.5 * frequency
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, frequency, order):
    b, a = butter_bandpass(lowcut, highcut, frequency, order)
    y = lfilter(b, a, data)
    return y

### Find Peaks

In [15]:
def findpeaks(data, spacing=1, limit=None):
        """
        Janko Slavic peak detection algorithm and implementation.
        https://github.com/jankoslavic/py-tools/tree/master/findpeaks
        Finds peaks in `data` which are of `spacing` width and >=`limit`.
        :param ndarray data: data
        :param float spacing: minimum spacing to the next peak (should be 1 or more)
        :param float limit: peaks should have value greater or equal
        :return array: detected peaks indexes array
        """
        len = data.size
        x = np.zeros(len + 2 * spacing)
        x[:spacing] = data[0] - 1.e-6
        x[-spacing:] = data[-1] - 1.e-6
        x[spacing:spacing + len] = data
        peak_candidate = np.zeros(len)
        peak_candidate[:] = True
        for s in range(spacing):
            start = spacing - s - 1
            h_b = x[start: start + len]  # before
            start = spacing
            h_c = x[start: start + len]  # central
            start = spacing + s + 1
            h_a = x[start: start + len]  # after
            peak_candidate = np.logical_and(peak_candidate, np.logical_and(h_c > h_b, h_c > h_a))

        ind = np.argwhere(peak_candidate)
        ind = ind.reshape(ind.size)
        if limit is not None:
            ind = ind[data[ind] > limit]
        return ind

### Detect Peaks

In [16]:
def detect_peaks(ecg_measurements,signal_frequency,gain):

        """
        Method responsible for extracting peaks from loaded ECG measurements data through measurements processing.
        This implementation of a QRS Complex Detector is by no means a certified medical tool and should not be used in health monitoring. 
        It was created and used for experimental purposes in psychophysiology and psychology.
        You can find more information in module documentation:
        https://github.com/c-labpl/qrs_detector
        If you use these modules in a research project, please consider citing it:
        https://zenodo.org/record/583770
        If you use these modules in any other project, please refer to MIT open-source license.
        If you have any question on the implementation, please refer to:
        Michal Sznajder (Jagiellonian University) - technical contact (msznajder@gmail.com)
        Marta lukowska (Jagiellonian University)
        Janko Slavic peak detection algorithm and implementation.
        https://github.com/c-labpl/qrs_detector
        https://github.com/jankoslavic/py-tools/tree/master/findpeaks
        
        MIT License
        Copyright (c) 2017 Michal Sznajder, Marta Lukowska
    
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
        IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
        FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
        AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
        LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
        OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
        SOFTWARE.
        """


        filter_lowcut = 0.001
        filter_highcut = 15.0
        filter_order = 1
        integration_window = 30  # Change proportionally when adjusting frequency (in samples).
        findpeaks_limit = 0.01 # changed from 0.35
        findpeaks_spacing = 100  # Change proportionally when adjusting frequency (in samples).
        refractory_period = 240  # Change proportionally when adjusting frequency (in samples).
        qrs_peak_filtering_factor = 0.125
        noise_peak_filtering_factor = 0.125
        qrs_noise_diff_weight = 0.25


        # Detection results.
        qrs_peaks_indices = np.array([], dtype=int)
        noise_peaks_indices = np.array([], dtype=int)

        # Derivative - provides QRS slope information.
        differentiated_ecg_measurements = np.ediff1d(ecg_measurements)

        # Squaring - intensifies values received in derivative.
        squared_ecg_measurements = differentiated_ecg_measurements ** 2

        # Moving-window integration.
        integrated_ecg_measurements = np.convolve(squared_ecg_measurements, np.ones(integration_window)/integration_window)

        # Fiducial mark - peak detection on integrated measurements.
        detected_peaks_indices = findpeaks(data=integrated_ecg_measurements,
                                                     limit=findpeaks_limit,
                                                     spacing=findpeaks_spacing)

        detected_peaks_values = integrated_ecg_measurements[detected_peaks_indices]

        return detected_peaks_values,detected_peaks_indices

### Fetching signals and feature extraction

In [14]:
def signals_features(diagnostics_df, num_records):
    
    def get_info(filename, age, sex, label):
        info = [filename, age, sex, label]
        return info

    info = [get_info(record[0], record[1], 
                     record[2], record[3]) for record in zip(diagnostics_df['filename'], diagnostics_df['age'], 
                                                  diagnostics_df['sex'], diagnostics_df['label'])]


    signals = np.zeros(shape=(num_records, 5000, 12))
    frequency = 500
    filter_lowcut = 0.001
    filter_highcut = 15.0
    filter_order = 1

    age_list = []
    sex_list = []
    label_list = []
    statistical_features = []

    i = 1
    print("Getting signals from records...")
    for record in info[0:num_records]:
        print(">>> Number " + str(i) + "/" + str(num_records))
        path = "../data/chapman/ECGDataDenoised/" + record[0] + ".csv"
        try:
            signals[i-1] = np.genfromtxt(path, delimiter=',')
            age_list.append(record[1])
            sex_list.append(record[2])
            label_list.append(record[3]) 
        except ValueError as e:
            pass
        i += 1
    
    print("Done")

    # get new num_records
    num_records = len(label_list)
    signals = signals[0:num_records]


    print("Getting statistical features from each lead...")

    for signal in signals:
        signal = signal.T
        signal = butter_bandpass_filter(signal, filter_lowcut, filter_highcut, frequency, filter_order)
        features = np.zeros(shape=(12,13))

        for j in range(12):

            # get the root mean square for each lead
            values = signal[j]
            rms = np.sqrt(np.mean(values**2)) 

            # get R peaks from signal
            peaks, idx = detect_peaks(values, 500, 1000)

            # get statistical features of the RR interval and R-peaks

            # mean
            mean_RR = np.mean(idx/frequency*1000)
            mean_Peaks = np.mean(peaks*1000)

            # median
            median_RR = np.median(idx/frequency*1000)
            median_Peaks = np.median(peaks*1000)

            # standard deviation
            std_RR = np.std(idx/frequency*1000)
            std_Peaks = np.std(peaks*1000)

            # variance
            var_RR = stats.tvar(idx/frequency*1000)
            var_Peaks = stats.tvar(peaks*1000)

            # skewness
            skew_RR = stats.skew(idx/frequency*1000)
            skew_Peaks = stats.skew(peaks*1000)

            # kurtosis
            kurt_RR = stats.kurtosis(idx/frequency*1000)
            kurt_Peaks = stats.kurtosis(peaks*1000)

            features[j] = [rms, mean_RR, mean_Peaks, median_RR, median_Peaks, std_RR, std_Peaks,
                        var_RR, var_Peaks, skew_RR, skew_Peaks, kurt_RR, kurt_Peaks]


        statistical_features.append(features)

    print("Done")

    dataset = pd.DataFrame(np.array(statistical_features).reshape(num_records,156), columns=columns)   
    dataset['age'], dataset['sex'], dataset['label'] = [age_list, sex_list, label_list]
    
    # drop null values
    dataset = dataset.dropna()
    
    # shuffle dataset
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    
    return dataset

In [15]:
columns = ['I_rms', 'I_mean_RR', 'I_mean_Peaks', 'I_median_RR', 'I_median_Peaks', 'I_std_RR', 'I_std_Peaks', 'I_var_RR', 'I_var_Peaks', 'I_skew_RR', 'I_skew_Peaks', 'I_kurt_RR', 'I_kurt_Peaks',
           'II_rms', 'II_mean_RR', 'II_mean_Peaks', 'II_median_RR', 'II_median_Peaks', 'II_std_RR', 'II_std_Peaks', 'II_var_RR', 'II_var_Peaks', 'II_skew_RR', 'II_skew_Peaks', 'II_kurt_RR', 'II_kurt_Peaks',
           'III_rms', 'III_mean_RR', 'III_mean_Peaks', 'III_median_RR', 'III_median_Peaks', 'III_std_RR', 'III_std_Peaks', 'III_var_RR', 'III_var_Peaks', 'III_skew_RR', 'III_skew_Peaks', 'III_kurt_RR', 'III_kurt_Peaks',
           'AVR_rms', 'AVR_mean_RR', 'AVR_mean_Peaks', 'AVR_median_RR', 'AVR_median_Peaks', 'AVR_std_RR', 'AVR_std_Peaks', 'AVR_var_RR', 'AVR_var_Peaks', 'AVR_skew_RR', 'AVR_skew_Peaks', 'AVR_kurt_RR', 'AVR_kurt_Peaks',
           'AVL_rms', 'AVL_mean_RR', 'AVL_mean_Peaks', 'AVL_median_RR', 'AVL_median_Peaks', 'AVL_std_RR', 'AVL_std_Peaks', 'AVL_var_RR', 'AVL_var_Peaks', 'AVL_skew_RR', 'AVL_skew_Peaks', 'AVL_kurt_RR', 'AVL_kurt_Peaks',
           'AVF_rms', 'AVF_mean_RR', 'AVF_mean_Peaks', 'AVF_median_RR', 'AVF_median_Peaks', 'AVF_std_RR', 'AVF_std_Peaks', 'AVF_var_RR', 'AVF_var_Peaks', 'AVF_skew_RR', 'AVF_skew_Peaks', 'AVF_kurt_RR', 'AVF_kurt_Peaks',
           'V1_rms', 'V1_mean_RR', 'V1_mean_Peaks', 'V1_median_RR', 'V1_median_Peaks', 'V1_std_RR', 'V1_std_Peaks', 'V1_var_RR', 'V1_var_Peaks', 'V1_skew_RR', 'V1_skew_Peaks', 'V1_kurt_RR', 'V1_kurt_Peaks',
           'V2_rms', 'V2_mean_RR', 'V2_mean_Peaks', 'V2_median_RR', 'V2_median_Peaks', 'V2_std_RR', 'V2_std_Peaks', 'V2_var_RR', 'V2_var_Peaks', 'V2_skew_RR', 'V2_skew_Peaks', 'V2_kurt_RR', 'V2_kurt_Peaks',
           'V3_rms', 'V3_mean_RR', 'V3_mean_Peaks', 'V3_median_RR', 'V3_median_Peaks', 'V3_std_RR', 'V3_std_Peaks', 'V3_var_RR', 'V3_var_Peaks', 'V3_skew_RR', 'V3_skew_Peaks', 'V3_kurt_RR', 'V3_kurt_Peaks',
           'V4_rms', 'V4_mean_RR', 'V4_mean_Peaks', 'V4_median_RR', 'V4_median_Peaks', 'V4_std_RR', 'V4_std_Peaks', 'V4_var_RR', 'V4_var_Peaks', 'V4_skew_RR', 'V4_skew_Peaks', 'V4_kurt_RR', 'V4_kurt_Peaks',
           'V5_rms', 'V5_mean_RR', 'V5_mean_Peaks', 'V5_median_RR', 'V5_median_Peaks', 'V5_std_RR', 'V5_std_Peaks', 'V5_var_RR', 'V5_var_Peaks', 'V5_skew_RR', 'V5_skew_Peaks', 'V5_kurt_RR', 'V5_kurt_Peaks',
           'V6_rms', 'V6_mean_RR', 'V6_mean_Peaks', 'V6_median_RR', 'V6_median_Peaks', 'V6_std_RR', 'V6_std_Peaks', 'V6_var_RR', 'V6_var_Peaks', 'V6_skew_RR', 'V6_skew_Peaks', 'V6_kurt_RR', 'V6_kurt_Peaks']


# Multiclass dataset: AF, Other, None

### Select records for each rhythm type and assign a label

In [16]:
# suppress chained assignment warning
pd.options.mode.chained_assignment = None

# sinus rhythm
chapman_sr = chapman.loc[chapman['rhythm'] == 'SR']
chapman_sr['label'] = 0

# atrial fibrillation
chapman_afib = chapman.loc[chapman['rhythm'] == 'AFIB']
chapman_afib['label'] = 1

# other rhythms
rhythms = ['SR','AFIB']
chapman_other = chapman[~chapman['rhythm'].isin(rhythms)]
chapman_other['label'] = 2

# combine dataframes
chapman_ = pd.concat([chapman_sr, chapman_afib, chapman_other])
chapman_

,filename,rhythm,age,sex,label
7,MUSE_20180209_172046_21000,SR,46,0,0
20,MUSE_20180209_122707_18000,SR,46,1,0
41,MUSE_20180210_121425_58000,SR,80,1,0
46,MUSE_20180209_123610_67000,SR,48,1,0
50,MUSE_20180209_174940_69000,SR,72,0,0
...,...,...,...,...,...
10641,MUSE_20181222_204306_99000,SVT,80,1,2
10642,MUSE_20181222_204309_22000,SVT,81,1,2
10643,MUSE_20181222_204310_31000,SVT,39,0,2
10644,MUSE_20181222_204312_58000,SVT,76,0,2


In [17]:
chapman_['label'].value_counts()

2    7040
0    1826
1    1780
Name: label, dtype: int64

In [18]:
chapman_['rhythm'].value_counts()

SB       3889
SR       1826
AFIB     1780
ST       1568
SVT       587
AF        445
SI        399
AT        121
AVNRT      16
AVRT        8
SAAWR       7
Name: rhythm, dtype: int64

### Fetch signals and extract features

In [19]:
dataset_three = signals_features(chapman_, len(chapman_))

Getting signals from records...
>>> Number 1/10646
>>> Number 2/10646
>>> Number 3/10646
>>> Number 4/10646
>>> Number 5/10646
>>> Number 6/10646
>>> Number 7/10646
>>> Number 8/10646
>>> Number 9/10646
>>> Number 10/10646
>>> Number 11/10646
>>> Number 12/10646
>>> Number 13/10646
>>> Number 14/10646
>>> Number 15/10646
>>> Number 16/10646
>>> Number 17/10646
>>> Number 18/10646
>>> Number 19/10646
>>> Number 20/10646
>>> Number 21/10646
>>> Number 22/10646
>>> Number 23/10646
>>> Number 24/10646
>>> Number 25/10646
>>> Number 26/10646
>>> Number 27/10646
>>> Number 28/10646
>>> Number 29/10646
>>> Number 30/10646
>>> Number 31/10646
>>> Number 32/10646
>>> Number 33/10646
>>> Number 34/10646
>>> Number 35/10646
>>> Number 36/10646
>>> Number 37/10646
>>> Number 38/10646
>>> Number 39/10646
>>> Number 40/10646
>>> Number 41/10646
>>> Number 42/10646
>>> Number 43/10646
>>> Number 44/10646
>>> Number 45/10646
>>> Number 46/10646
>>> Number 47/10646
>>> Number 48/10646
>>> Number 49/106

>>> Number 401/10646
>>> Number 402/10646
>>> Number 403/10646
>>> Number 404/10646
>>> Number 405/10646
>>> Number 406/10646
>>> Number 407/10646
>>> Number 408/10646
>>> Number 409/10646
>>> Number 410/10646
>>> Number 411/10646
>>> Number 412/10646
>>> Number 413/10646
>>> Number 414/10646
>>> Number 415/10646
>>> Number 416/10646
>>> Number 417/10646
>>> Number 418/10646
>>> Number 419/10646
>>> Number 420/10646
>>> Number 421/10646
>>> Number 422/10646
>>> Number 423/10646
>>> Number 424/10646
>>> Number 425/10646
>>> Number 426/10646
>>> Number 427/10646
>>> Number 428/10646
>>> Number 429/10646
>>> Number 430/10646
>>> Number 431/10646
>>> Number 432/10646
>>> Number 433/10646
>>> Number 434/10646
>>> Number 435/10646
>>> Number 436/10646
>>> Number 437/10646
>>> Number 438/10646
>>> Number 439/10646
>>> Number 440/10646
>>> Number 441/10646
>>> Number 442/10646
>>> Number 443/10646
>>> Number 444/10646
>>> Number 445/10646
>>> Number 446/10646
>>> Number 447/10646
>>> Number 44

>>> Number 798/10646
>>> Number 799/10646
>>> Number 800/10646
>>> Number 801/10646
>>> Number 802/10646
>>> Number 803/10646
>>> Number 804/10646
>>> Number 805/10646
>>> Number 806/10646
>>> Number 807/10646
>>> Number 808/10646
>>> Number 809/10646
>>> Number 810/10646
>>> Number 811/10646
>>> Number 812/10646
>>> Number 813/10646
>>> Number 814/10646
>>> Number 815/10646
>>> Number 816/10646
>>> Number 817/10646
>>> Number 818/10646
>>> Number 819/10646
>>> Number 820/10646
>>> Number 821/10646
>>> Number 822/10646
>>> Number 823/10646
>>> Number 824/10646
>>> Number 825/10646
>>> Number 826/10646
>>> Number 827/10646
>>> Number 828/10646
>>> Number 829/10646
>>> Number 830/10646
>>> Number 831/10646
>>> Number 832/10646
>>> Number 833/10646
>>> Number 834/10646
>>> Number 835/10646
>>> Number 836/10646
>>> Number 837/10646
>>> Number 838/10646
>>> Number 839/10646
>>> Number 840/10646
>>> Number 841/10646
>>> Number 842/10646
>>> Number 843/10646
>>> Number 844/10646
>>> Number 84

>>> Number 1182/10646
>>> Number 1183/10646
>>> Number 1184/10646
>>> Number 1185/10646
>>> Number 1186/10646
>>> Number 1187/10646
>>> Number 1188/10646
>>> Number 1189/10646
>>> Number 1190/10646
>>> Number 1191/10646
>>> Number 1192/10646
>>> Number 1193/10646
>>> Number 1194/10646
>>> Number 1195/10646
>>> Number 1196/10646
>>> Number 1197/10646
>>> Number 1198/10646
>>> Number 1199/10646
>>> Number 1200/10646
>>> Number 1201/10646
>>> Number 1202/10646
>>> Number 1203/10646
>>> Number 1204/10646
>>> Number 1205/10646
>>> Number 1206/10646
>>> Number 1207/10646
>>> Number 1208/10646
>>> Number 1209/10646
>>> Number 1210/10646
>>> Number 1211/10646
>>> Number 1212/10646
>>> Number 1213/10646
>>> Number 1214/10646
>>> Number 1215/10646
>>> Number 1216/10646
>>> Number 1217/10646
>>> Number 1218/10646
>>> Number 1219/10646
>>> Number 1220/10646
>>> Number 1221/10646
>>> Number 1222/10646
>>> Number 1223/10646
>>> Number 1224/10646
>>> Number 1225/10646
>>> Number 1226/10646
>>> Number

>>> Number 1558/10646
>>> Number 1559/10646
>>> Number 1560/10646
>>> Number 1561/10646
>>> Number 1562/10646
>>> Number 1563/10646
>>> Number 1564/10646
>>> Number 1565/10646
>>> Number 1566/10646
>>> Number 1567/10646
>>> Number 1568/10646
>>> Number 1569/10646
>>> Number 1570/10646
>>> Number 1571/10646
>>> Number 1572/10646
>>> Number 1573/10646
>>> Number 1574/10646
>>> Number 1575/10646
>>> Number 1576/10646
>>> Number 1577/10646
>>> Number 1578/10646
>>> Number 1579/10646
>>> Number 1580/10646
>>> Number 1581/10646
>>> Number 1582/10646
>>> Number 1583/10646
>>> Number 1584/10646
>>> Number 1585/10646
>>> Number 1586/10646
>>> Number 1587/10646
>>> Number 1588/10646
>>> Number 1589/10646
>>> Number 1590/10646
>>> Number 1591/10646
>>> Number 1592/10646
>>> Number 1593/10646
>>> Number 1594/10646
>>> Number 1595/10646
>>> Number 1596/10646
>>> Number 1597/10646
>>> Number 1598/10646
>>> Number 1599/10646
>>> Number 1600/10646
>>> Number 1601/10646
>>> Number 1602/10646
>>> Number

...

>>> Number 9815/10646
>>> Number 9816/10646
>>> Number 9817/10646
>>> Number 9818/10646
>>> Number 9819/10646
>>> Number 9820/10646
>>> Number 9821/10646
>>> Number 9822/10646
>>> Number 9823/10646
>>> Number 9824/10646
>>> Number 9825/10646
>>> Number 9826/10646
>>> Number 9827/10646
>>> Number 9828/10646
>>> Number 9829/10646
>>> Number 9830/10646
>>> Number 9831/10646
>>> Number 9832/10646
>>> Number 9833/10646
>>> Number 9834/10646
>>> Number 9835/10646
>>> Number 9836/10646
>>> Number 9837/10646
>>> Number 9838/10646
>>> Number 9839/10646
>>> Number 9840/10646
>>> Number 9841/10646
>>> Number 9842/10646
>>> Number 9843/10646
>>> Number 9844/10646
>>> Number 9845/10646
>>> Number 9846/10646
>>> Number 9847/10646
>>> Number 9848/10646
>>> Number 9849/10646
>>> Number 9850/10646
>>> Number 9851/10646
>>> Number 9852/10646
>>> Number 9853/10646
>>> Number 9854/10646
>>> Number 9855/10646
>>> Number 9856/10646
>>> Number 9857/10646
>>> Number 9858/10646
>>> Number 9859/10646
>>> Number

>>> Number 10180/10646
>>> Number 10181/10646
>>> Number 10182/10646
>>> Number 10183/10646
>>> Number 10184/10646
>>> Number 10185/10646
>>> Number 10186/10646
>>> Number 10187/10646
>>> Number 10188/10646
>>> Number 10189/10646
>>> Number 10190/10646
>>> Number 10191/10646
>>> Number 10192/10646
>>> Number 10193/10646
>>> Number 10194/10646
>>> Number 10195/10646
>>> Number 10196/10646
>>> Number 10197/10646
>>> Number 10198/10646
>>> Number 10199/10646
>>> Number 10200/10646
>>> Number 10201/10646
>>> Number 10202/10646
>>> Number 10203/10646
>>> Number 10204/10646
>>> Number 10205/10646
>>> Number 10206/10646
>>> Number 10207/10646
>>> Number 10208/10646
>>> Number 10209/10646
>>> Number 10210/10646
>>> Number 10211/10646
>>> Number 10212/10646
>>> Number 10213/10646
>>> Number 10214/10646
>>> Number 10215/10646
>>> Number 10216/10646
>>> Number 10217/10646
>>> Number 10218/10646
>>> Number 10219/10646
>>> Number 10220/10646
>>> Number 10221/10646
>>> Number 10222/10646
>>> Number 

>>> Number 10544/10646
>>> Number 10545/10646
>>> Number 10546/10646
>>> Number 10547/10646
>>> Number 10548/10646
>>> Number 10549/10646
>>> Number 10550/10646
>>> Number 10551/10646
>>> Number 10552/10646
>>> Number 10553/10646
>>> Number 10554/10646
>>> Number 10555/10646
>>> Number 10556/10646
>>> Number 10557/10646
>>> Number 10558/10646
>>> Number 10559/10646
>>> Number 10560/10646
>>> Number 10561/10646
>>> Number 10562/10646
>>> Number 10563/10646
>>> Number 10564/10646
>>> Number 10565/10646
>>> Number 10566/10646
>>> Number 10567/10646
>>> Number 10568/10646
>>> Number 10569/10646
>>> Number 10570/10646
>>> Number 10571/10646
>>> Number 10572/10646
>>> Number 10573/10646
>>> Number 10574/10646
>>> Number 10575/10646
>>> Number 10576/10646
>>> Number 10577/10646
>>> Number 10578/10646
>>> Number 10579/10646
>>> Number 10580/10646
>>> Number 10581/10646
>>> Number 10582/10646
>>> Number 10583/10646
>>> Number 10584/10646
>>> Number 10585/10646
>>> Number 10586/10646
>>> Number 

/Users/mbithe/.virtualenvs/health_agent/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/mbithe/.virtualenvs/health_agent/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/mbithe/.virtualenvs/health_agent/lib/python3.9/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/mbithe/.virtualenvs/health_agent/lib/python3.9/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/Users/mbithe/.virtualenvs/health_agent/lib/python3.9/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/mbithe/.virtualenvs/health_agent

Done


In [20]:
dataset_three.shape

(10588, 159)

In [21]:
dataset_three['label'].value_counts()

2    6983
0    1825
1    1780
Name: label, dtype: int64

In [22]:
# balance dataset
af = dataset_three.loc[dataset_three['label'] == 1]
other = dataset_three.loc[dataset_three['label'] == 2]
other = other.sample(len(af), random_state=1)
none = dataset_three.loc[dataset_three['label'] == 0]
none = none.sample(len(af), random_state=1)
dataset_three = pd.concat([af, other, none])
dataset_three['label'].value_counts()

1    1780
2    1780
0    1780
Name: label, dtype: int64

In [23]:
# save the dataset
dataset_three.to_csv('data/dataset_chapman.csv', index=False)

### Summary stats

In [24]:
dataset_three.describe()

,I_rms,I_mean_RR,I_mean_Peaks,I_median_RR,I_median_Peaks,I_std_RR,I_std_Peaks,I_var_RR,I_var_Peaks,I_skew_RR,...,V6_std_Peaks,V6_var_RR,V6_var_Peaks,V6_skew_RR,V6_skew_Peaks,V6_kurt_RR,V6_kurt_Peaks,age,sex,label
count,5340.000000,5340.000000,5.340000e+03,5340.000000,5.340000e+03,5340.000000,5.340000e+03,5.340000e+03,5.340000e+03,5340.000000,...,5.340000e+03,5.340000e+03,5.340000e+03,5340.000000,5340.000000,5340.000000,5340.000000,5340.000000,5340.000000,5340.000000
mean,90.080329,5020.603193,9.735153e+05,5018.675843,8.950564e+05,2954.571823,6.486944e+05,9.165014e+06,1.946620e+12,0.000418,...,3.126342e+06,9.166542e+06,2.816869e+13,-0.000933,-0.445753,-1.202068,0.181010,61.408052,0.467790,1.000000
std,45.444469,194.747729,1.058093e+06,314.699291,1.230889e+06,91.447982,1.196466e+06,5.964427e+05,5.447893e+13,0.086656,...,4.132991e+06,6.099860e+05,1.700941e+14,0.082464,1.227334,0.067943,4.228178,17.770076,0.499008,0.816573
min,13.166587,3745.764706,9.256109e+03,2982.000000,5.166767e+02,2613.817987,1.816779e+03,7.157380e+06,3.385319e+06,-0.398722,...,6.706595e+03,6.966936e+06,4.734570e+07,-0.429309,-5.383309,-1.486271,-1.999940,4.000000,0.000000,0.000000
25%,60.754309,4907.744589,3.383997e+05,4852.000000,6.499269e+04,2893.537546,1.965826e+05,8.760942e+06,4.056607e+10,-0.035523,...,1.053217e+06,8.761999e+06,1.168056e+12,-0.034416,-0.774522,-1.234470,-1.956852,51.000000,0.000000,0.000000
50%,82.052445,5027.718944,6.797576e+05,5027.000000,4.871753e+05,2952.363678,4.297096e+05,9.125414e+06,1.942227e+11,0.000030,...,2.058785e+06,9.116015e+06,4.449797e+12,-0.000446,0.000670,-1.207550,-1.824429,64.000000,0.000000,1.000000
75%,109.969621,5134.759740,1.245516e+06,5186.000000,1.261730e+06,3009.383852,8.609518e+05,9.517132e+06,7.763921e+11,0.038141,...,3.619543e+06,9.517827e+06,1.379561e+13,0.032046,0.131986,-1.177192,-0.061719,75.000000,1.000000,2.000000
max,555.140591,5798.476190,2.459256e+07,6494.000000,2.451293e+07,3397.097519,5.424419e+07,1.221911e+07,3.097297e+15,0.507490,...,8.892608e+07,1.237970e+07,8.251667e+15,0.418612,5.196805,-0.835003,26.998580,98.000000,1.000000,2.000000


### Check distribution between genders

In [25]:
dataset_three['sex'].value_counts()

0    2842
1    2498
Name: sex, dtype: int64

### Get summary stats on age

In [26]:
dataset_three['age'].describe()

count    5340.000000
mean       61.408052
std        17.770076
min         4.000000
25%        51.000000
50%        64.000000
75%        75.000000
max        98.000000
Name: age, dtype: float64

# Binary dataset  (AF, None)

In [27]:
af_none = dataset_three[dataset_three['label'] != 2]
af_none.shape

(3560, 159)

In [28]:
af_none['age'].describe()

count    3560.000000
mean       63.876966
std        16.882385
min         5.000000
25%        53.000000
50%        67.000000
75%        77.000000
max        98.000000
Name: age, dtype: float64

In [29]:
af_none['sex'].value_counts()

0    1826
1    1734
Name: sex, dtype: int64